RAG with Chroma and GPT3.5

In [1]:
from langchain.document_loaders import PyPDFLoader,CSVLoader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter, RecursiveCharacterTextSplitter
from langchain_openai import ChatOpenAI


In [2]:
openai_key=''

In [3]:
from langchain_openai import OpenAIEmbeddings,OpenAI
llm = OpenAI(model_name="gpt-3.5-turbo-instruct", openai_api_key=openai_key)

In [4]:
loader = PyPDFLoader(file_path='Data/RAG_Documents/document.pdf')
data = loader.load()

chunk_size = 200
chunk_overlap = 50
rc_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size,chunk_overlap=chunk_overlap)


Ignoring wrong pointing object 13 0 (offset 0)


In [5]:
docs = rc_splitter.split_documents(data)

In [6]:
#Embedding the data and saving to a vector db
from langchain_openai import OpenAIEmbeddings,OpenAI
from langchain_community.vectorstores import Chroma

embedding_function = OpenAIEmbeddings(openai_api_key=openai_key)
chromadb = Chroma(persist_directory='Data/Chroma/',embedding_function=embedding_function)
chromadb.persist()
docstorage = Chroma.from_documents(docs, embedding_function)

In [7]:
db = docstorage
print(db.get().keys())
print(len(db.get()["ids"]))

#db.get()['documents']
# # Print the list of source files
# for x in range(len(db.get()["ids"])):
#     # print(db.get()["metadatas"][x])
#     doc = db.get()["metadatas"][x]
#     source = doc["source"]
#     print(source)

dict_keys(['ids', 'embeddings', 'metadatas', 'documents', 'uris', 'data'])
93


In [8]:
#querying the db
query = 'what is the primary architecture presented in the document?'
result = chromadb.similarity_search_with_relevance_scores(query)
result

[]

In [9]:
from langchain.chains import RetrievalQA

retriever = docstorage.as_retriever()
qa = RetrievalQA.from_llm(llm=llm, retriever=retriever)

query='what is the primary architecture presented in the document?'
results = qa.invoke(query)

print(results)

{'query': 'what is the primary architecture presented in the document?', 'result': '\nThe primary architecture presented in the document is a visual representation of how things are connected, with AI on top and other components branching out. It is generated using Microsoft Designer.'}


In [11]:
#Showing reference to the document in the response
from langchain.chains import RetrievalQAWithSourcesChain

qa_source = RetrievalQAWithSourcesChain.from_chain_type(llm=llm, retriever=retriever)

results = qa_source.invoke(query,return_only_outputs=True)
print(results)

{'answer': ' The primary architecture presented in the document is a comparison between human and machine cognition, with AI at the top and other components branching out. The document also includes a visual representation of how things are connected, and was generated using Microsoft Designer.\n', 'sources': 'Data/RAG_Documents/document.pdf'}
